In [1]:
import pickle
import tensorflow
import numpy as np
from numpy.linalg import norm

from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors

In [2]:
import cv2


In [3]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

In [4]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])
print(model.summary())


Model: "sequential"
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Glob  (None, 2048)              0         
 alMaxPooling2D)                                                 
                                                                 
                                                                 
 global_max_pooling2d (Glob  (None, 2048)              0         
 alMaxPooling2D)                                                 
                                                                 
Total params: 23587712 (89.98 MB)
Tr

In [5]:
feature_list=pickle.load(open('embeddings.pkl','rb'))
print(feature_list)

[array([0.01886855, 0.01673299, 0.003997  , ..., 0.00810945, 0.01061399,
       0.05708206], dtype=float32), array([0.        , 0.02039722, 0.        , ..., 0.        , 0.        ,
       0.04172791], dtype=float32), array([0.00055714, 0.03378946, 0.01351393, ..., 0.        , 0.        ,
       0.02299199], dtype=float32), array([0.01809278, 0.06029446, 0.00208155, ..., 0.01785667, 0.02855489,
       0.06173433], dtype=float32), array([0.02202039, 0.01658256, 0.        , ..., 0.00803621, 0.0213541 ,
       0.02802395], dtype=float32), array([0.        , 0.11093646, 0.01128945, ..., 0.        , 0.01216864,
       0.02503347], dtype=float32), array([0.00928833, 0.10958564, 0.00514373, ..., 0.00458986, 0.03560196,
       0.03131818], dtype=float32), array([0.        , 0.1242571 , 0.00651039, ..., 0.        , 0.00952099,
       0.03090211], dtype=float32), array([0.006378  , 0.09182654, 0.00335865, ..., 0.00216756, 0.0224888 ,
       0.03567941], dtype=float32), array([0.01277727, 0.103790

[array([0.01886855, 0.01673299, 0.003997  , ..., 0.00810945, 0.01061399,
       0.05708206], dtype=float32), array([0.        , 0.02039722, 0.        , ..., 0.        , 0.        ,
       0.04172791], dtype=float32), array([0.00055714, 0.03378946, 0.01351393, ..., 0.        , 0.        ,
       0.02299199], dtype=float32), array([0.01809278, 0.06029446, 0.00208155, ..., 0.01785667, 0.02855489,
       0.06173433], dtype=float32), array([0.02202039, 0.01658256, 0.        , ..., 0.00803621, 0.0213541 ,
       0.02802395], dtype=float32), array([0.        , 0.11093646, 0.01128945, ..., 0.        , 0.01216864,
       0.02503347], dtype=float32), array([0.00928833, 0.10958564, 0.00514373, ..., 0.00458986, 0.03560196,
       0.03131818], dtype=float32), array([0.        , 0.1242571 , 0.00651039, ..., 0.        , 0.00952099,
       0.03090211], dtype=float32), array([0.006378  , 0.09182654, 0.00335865, ..., 0.00216756, 0.0224888 ,
       0.03567941], dtype=float32), array([0.01277727, 0.103790

In [6]:
filenames=pickle.load(open('filenames.pkl','rb'))

In [7]:

    img = image.load_img('sample.jpg',target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

  

1/1 [==============================] - 2s 2s/step


In [8]:
neighbors= NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')


In [9]:
neighbors.fit(feature_list)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [10]:
distances,indices = neighbors.kneighbors([normalized_result])
print(indices)

[[6959 4532 7348 8664 5292 5684]]
[[6959 4532 7348 8664 5292 5684]]


In [11]:
for file in indices[0][1:6]:
    temp_img = cv2.imread(filenames[file])
    resized_image = cv2.resize(temp_img, (360, 380), interpolation=cv2.INTER_CUBIC)

    cv2.imshow('output',cv2.resize(temp_img,(360,380)))
    cv2.waitKey(0)